In [1]:
include("../generalized-chart-parser/parser_types.jl")

Main.ParserTypes

In [2]:
using LogProbs
using StatsFuns.RFunctions: gammarand
using StatsFuns: lbeta
import Base: +, -, *, /, zero, one, <

In [3]:
###############
### CFRules ###
###############

mutable struct RunningCounter
    n :: Int
end

RunningCounter() = RunningCounter(0)
count!(c::RunningCounter) = c.n += 1

rule_counter = RunningCounter()

struct CFRules{LHS, RHS} # left hand side and right hand side of the rule
    mappings ::Dict{LHS, Vector{RHS}}
    name :: Symbol
end

==(r1::CFRules, r2::CFRules) = r1.name == r2.name
hash(r::CFRules, h::UInt) = hash(hash(CFRules, hash(r.name)), h)

Base.show(io::IO, r::CFRules) = print(io, "CFRules($(r.name))")

CFRules(pairs::Pair...) =
    CFRules(Dict(pairs...), Symbol("rules", count!(rule_counter)))
CFRules(g::Base.Generator) =
    CFRules(Dict(g), Symbol("rules", count!(rule_counter)))
CFRules(f::Function, lhss, name) =
    CFRules(Dict(lhs => f(lhs) for lhs in lhss), name)
CFRules(f::Function, lhss) =
    CFRules(Dict(lhs => f(lhs) for lhs in lhss), Symbol("rules", count!(rule_counter)))

lhss(r::CFRules) = keys(r.mappings) # aka domain
isapplicable(r::CFRules, lhs) = haskey(r.mappings, lhs)
(r::CFRules)(lhs) = r.mappings[lhs]

In [5]:
###############
### CFState ###
###############


mutable struct CompletionAutomaton{Cat,Comp} # category, completion
    transitions :: Vector{Dict{Cat, Int}}
    completions :: Vector{Vector{Comp}}
end

CompletionAutomaton(Cat::Type, Comp::Type) =
    CompletionAutomaton([Dict{Cat, Int}()], [Vector{Comp}()])

number_of_states(ca::CompletionAutomaton) = length(ca.transitions)
isfinal(ca::CompletionAutomaton, s) = isempty(ca.transitions[s])
is_possible_transition(ca::CompletionAutomaton, s, c) = haskey(ca.transitions[s], c)
transition(ca::CompletionAutomaton, s, c) = ca.transitions[s][c]
completions(ca::CompletionAutomaton, s) = ca.completions[s]

#Not sure what is going on here
function add_completion!(ca::CompletionAutomaton{Cat,Comp}, comp, categories) where {Cat,Comp}
    s = 1
    for c in categories
        if is_possible_transition(ca, s, c)
            s = transition(ca, s, c)
        else
            push!(ca.transitions, Dict{Cat,Int}())
            push!(ca.completions, Vector{Comp}())
            s = ca.transitions[s][c] = number_of_states(ca)
        end
    end
    push!(ca.completions[s], comp)
end

function add_rule!(ca::CompletionAutomaton, r::CFRules)
    for lhs in lhss(r)
        add_completion!(ca, (lhs, r), r(lhs))
    end
end

add_rule! (generic function with 1 method)

In [7]:
#################
### CFGrammar ###
#################

struct CFGrammar{C, T, Cond, F}
    comp_automtn  :: CompletionAutomaton{C, Tuple{C, CFRule{C, C}}}
    startsymbols  :: Vector{C}
    terminal_dict :: Dict{T, Vector{Tuple{C, CFRule{C, T}}}}
    cond          :: Cond # conditional scoring
    dependent_components::F
end


In [8]:
function CFGrammar(
        category_rules::Vector{CFRule{C, C}},
        terminal_rules::Vector{CFRule{C, T}},
        startsymbols  ::Vector{C},
        dependent_components=identity::Function
    ) where {C, T}
    comp_automtn = CompletionAutomaton(C, Tuple{C, CFRule{C, C}})
    for r in category_rules
        add_rule!(comp_automtn, r)
    end

    terminal_dict = Dict{T, Vector{Tuple{C, CFRule{C, T}}}}()
    for r in terminal_rules
        for lhs in lhss(r)
        t = r(lhs)[1]
            if haskey(terminal_dict, t)
                push!(terminal_dict[t], (lhs, r))
            else
                terminal_dict[t] = [(lhs, r)]
            end
        end
    end

    applicable_rules = Dict{C, Vector{CFRule}}()
    for r in CFRule[category_rules; terminal_rules]
        for c in lhss(r)
            if haskey(applicable_rules, c)
                push!(applicable_rules[c], r)
            else
                applicable_rules[c] = CFRule[r]
            end
        end
    end

    cond = SimpleCond(
        Dict(
            dependent_components(c) => let rules = applicable_rules[c]
                n = length(rules)
                k = count(isa.(rules, CFRule{C, T})) # number terminal rules
                DirCat(rules, [fill(1.0, n-k); fill(1/k, k)])
            end
            for c in keys(applicable_rules)
        )
    )

    CFGrammar(comp_automtn, startsymbols, terminal_dict, cond, dependent_components)
end


CFGrammar